¿Qué Es el Web Scraping?
El web scraping es un conjunto de prácticas utilizadas para extraer automáticamente — o «scrapear» — datos de la web.

El web scraping se refiere al proceso de extracción de contenidos y datos de sitios web mediante software.

Este script realiza web scraping de la página de un libro específico en el sitio "Books to Scrape".

Primero, hacemos una petición HTTP con `requests.get(url)` para obtener el contenido HTML de la página.

Luego, usamos `BeautifulSoup(response.text, 'html.parser')` para convertir ese HTML crudo en un objeto navegable.
Esto permite buscar elementos fácilmente usando métodos como `.find()`, `.select_one()`, etc.

Extraemos el título con `soup.find('h1').text`, el precio con `soup.select_one('.price_color').text`,  y lo limpiamos con `.replace()` para quitar caracteres no deseados (como "Â").

Para la disponibilidad, usamos `.select_one('.availability').text.strip()` para eliminar espacios en blanco.

La descripción del libro está justo después del div con id `product_description`, por eso usamos `.find_next_sibling('p')`.

Finalmente, las estrellas están codificadas como una clase CSS en la etiqueta <p>, como por ejemplo: `<p class="star-rating Three">.`

Al usar `tag['class'][1]`, obtenemos el texto que indica la cantidad de estrellas (por ejemplo, "Three").

In [ ]:
# Extraer la información de un solo libro

import requests
from bs4 import BeautifulSoup #Averiguar bien

# URL del libro
url = 'https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'

# Hacemos la petición
response = requests.get(url)

# BeautifulSoup(response.text, 'html.parser') transforma el HTML crudo en un objeto navegable.
soup = BeautifulSoup(response.text, 'html.parser')

# Obtenemos el título
title = soup.find('h1').text

# .text te devuelve el texto visible dentro de una etiqueta HTML.
# La gente usa .text sin saber qué devuelve la etiqueta. Si la etiqueta no tiene texto visible (ej: <img> o <meta>), .text te da vacío.

# Obtenemos precio
price = soup.select_one('.price_color').text
price = price.replace('Â', '')  # Limpia cualquier carácter raro que aparezca

# Obtenemos disponibilidad
availability = soup.select_one('.availability').text.strip()

# Descripción (viene en el siguiente <p> después del div con id 'product_description')
desc_tag = soup.find('div', id='product_description')
# Usamos `.find_next_sibling('p')`, que busca el **próximo hermano (sibling)** que sea una etiqueta <p>.
description = desc_tag.find_next_sibling('p').text if desc_tag else "Sin descripción"

# Las estrellas están codificadas en la clase CSS de un <p>
# Ese guion bajo es una convención para evitar conflicto. Internamente, BeautifulSoup ya sabe que class_ se refiere al atributo class del HTML
# Al hacer tag['class'], obtenemos una lista como ['star-rating', 'Three'], y el índice [1] nos da la cantidad de estrellas en texto.
star_tag = soup.find('p', class_='star-rating')
stars = star_tag['class'][1]

# Imprimimos
print(f"Título: {title}")
print(f"Precio: {price}")
print(f"Disponibilidad: {availability}")
print(f"Descripción: {description[:60]}...")
print(f"Estrellas: {stars}")


Título: A Light in the Attic
Precio: £51.77
Disponibilidad: In stock (22 available)
Descripción: It's hard to imagine a world without A Light in the Attic. T...
Estrellas: Three


Scrapeo de varias páginas

Que hacen:

1. requests.get(url) hace una petición HTTP a la URL y devuelve la respuesta con el HTML.

2. BeautifulSoup transforma el texto HTML en un objeto para buscar etiquetas fácilmente.

3. .find() busca la primera etiqueta que cumple la condición.

4. .select() busca todas las etiquetas que cumplen un selector CSS (devuelve lista).

5. .select_one() busca el primer elemento que cumple el selector CSS (devuelve un tag).

6. En las URLs, a veces hay rutas relativas con "../", hay que limpiarlas para crear URLs absolutas.

7. time.sleep(0.1) hace que el programa espere 0.1 segundos para no saturar el servidor y evitar bloqueos.

8. El bucle while True se usa para seguir scrapeando páginas hasta que no haya más páginas siguientes.

9. Las estrellas del libro están en la clase CSS 'star-rating X', donde X es la cantidad en texto (One, Two, Three, etc).

10. Guardamos los datos en una lista de diccionarios para luego exportarlos o procesarlos fácilmente.

In [12]:
# Extraer la información de una categoría

import requests # Para hacer peticiones HTTPS y obtener el contenido de las páginas
from bs4 import BeautifulSoup # Para parsear (analizar) el contenido de la página web
import time # Para pausar entre peticiones y no sobrecargar el servidor

base_url = 'https://books.toscrape.com/catalogue/'

category_url = 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html'

# Lista donde se guardaran los datos de todos los libros
all_books = []

# Función scrape_book, toma la url de un libro especifico y devuelve los datos scrapeados en forma de diccionario
def scrape_book(url):

  # Hacemos la petición
  try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
  except requests.RequestException as e:
    print(f"❌ Error al acceder a {url}: {e}")
    return None

  # BeautifulSoup(response.text, 'html.parser') transforma el HTML crudo en un objeto navegable.
  soup = BeautifulSoup(response.text, 'html.parser')

  # Obtenemos el título
  title = soup.find('h1').text

  # Obtenemos precio
  price = soup.select_one('.price_color').text
  price = price.replace('Â', '')  # Limpia cualquier carácter raro que aparezca

  # Obtenemos disponibilidad
  availability = soup.select_one('.availability').text.strip()

  # Descripción (viene en el siguiente <p> después del div con id 'product_description')
  desc_tag = soup.find('div', id='product_description')
  # Usamos `.find_next_sibling('p')`, que busca el **próximo hermano (sibling)** que sea una etiqueta <p>.
  description = desc_tag.find_next_sibling('p').text if desc_tag else "Sin descripción"

  # Las estrellas están codificadas en la clase CSS de un <p>
  # Ese guion bajo es una convención para evitar conflicto. Internamente, BeautifulSoup ya sabe que class_ se refiere al atributo class del HTML
  # Al hacer tag['class'], obtenemos una lista como ['star-rating', 'Three'], y el índice [1] nos da la cantidad de estrellas en texto.
  star_tag = soup.find('p', class_='star-rating')
  stars = star_tag['class'][1]

  return {
    'Titulo': title,
    'Precio': price,
    'Disponibilidad': availability,
    'Estrellas': stars,
    'URL': url
  }

# Funcion scrape_category, toma la url de una categoria, recorre todas sus paginas y scrapea todos los libros que encuentra
def scrape_category(url):
  while True:
    print(f"Scrapeando pagina: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Seleccionamos todos los libros en la pagina actual de esa categoría
    book_articles = soup.select('article.product_pod')

    for book in book_articles:

      # Obtenemos el href del libro
      relative_url = book.find('h3').find('a')['href']

      # relative_url trae la url completa, entonces la limpiamos para después concatenarla a base_url
      book_url = base_url + relative_url.replace('../../../', '')

      # Scrapeamos el libro
      book_data = scrape_book(book_url)

      # Guardamos el resultado
      all_books.append(book_data)

      # Pausa para evitar sobrecarga
      time.sleep(10)

    # Verificamos si hay un boton next para seguir scrapeando la sgte pagina
    # Usa un selector CSS (li.next > a) para encontrar el link de la siguiente página. Si existe: next_button va a tener un tag <a>.
    next_button = soup.select_one('li.next > a')
    if next_button:
      next_page_url = next_button['href']
      # rsplit(separador, cantidad) divide un string desde el final, y solo hace la cantidad de cortes que vos le indiques.
      url = url.rsplit('/', 1)[0] + '/' + next_page_url
    else:
      break

scrape_category(category_url)

print(f"Total de libros scrapeados: {len(all_books)}")
print(all_books)

Scrapeando pagina: https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
❌ Error al acceder a https://books.toscrape.com/catalogue/the-last-mile-amos-decker-2_754/index.html: HTTPSConnectionPool(host='books.toscrape.com', port=443): Max retries exceeded with url: /catalogue/the-last-mile-amos-decker-2_754/index.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7d853d547080>, 'Connection to books.toscrape.com timed out. (connect timeout=10)'))
❌ Error al acceder a https://books.toscrape.com/catalogue/a-study-in-scarlet-sherlock-holmes-1_656/index.html: HTTPSConnectionPool(host='books.toscrape.com', port=443): Max retries exceeded with url: /catalogue/a-study-in-scarlet-sherlock-holmes-1_656/index.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7d853d9bbce0>, 'Connection to books.toscrape.com timed out. (connect timeout=10)'))
❌ Error al acceder a https://books.toscrape.com/catalogue/hide-away-eve-du

In [17]:
# Extraer información de toda la página

import requests # Para hacer peticiones HTTPS y obtener el contenido de las páginas
from bs4 import BeautifulSoup # Para parsear (analizar) el contenido de la página web
import time # Para pausar entre peticiones y no sobrecargar el servidor

base_url = 'https://books.toscrape.com/catalogue/'

# Lista donde se guardaran los datos de todos los libros
all_books = []

# Función scrape_book, toma la url de un libro especifico y devuelve los datos scrapeados en forma de diccionario
def scrape_book(url):

  # Hacemos la petición
  try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
  except requests.RequestException as e:
    print(f"❌ Error al acceder a {url}: {e}")
    return None

  # BeautifulSoup(response.text, 'html.parser') transforma el HTML crudo en un objeto navegable.
  soup = BeautifulSoup(response.text, 'html.parser')

  # Obtenemos el título
  title = soup.find('h1').text

  # Obtenemos precio
  price = soup.select_one('.price_color').text
  price = price.replace('Â', '')  # Limpia cualquier carácter raro que aparezca

  # Obtenemos disponibilidad
  availability = soup.select_one('.availability').text.strip()

  # Descripción (viene en el siguiente <p> después del div con id 'product_description')
  desc_tag = soup.find('div', id='product_description')
  # Usamos `.find_next_sibling('p')`, que busca el **próximo hermano (sibling)** que sea una etiqueta <p>.
  description = desc_tag.find_next_sibling('p').text if desc_tag else "Sin descripción"

  # Las estrellas están codificadas en la clase CSS de un <p>
  # Ese guion bajo es una convención para evitar conflicto. Internamente, BeautifulSoup ya sabe que class_ se refiere al atributo class del HTML
  # Al hacer tag['class'], obtenemos una lista como ['star-rating', 'Three'], y el índice [1] nos da la cantidad de estrellas en texto.
  star_tag = soup.find('p', class_='star-rating')
  stars = star_tag['class'][1]

  return {
    'Titulo': title,
    'Precio': price,
    'Disponibilidad': availability,
    'Estrellas': stars,
    'URL': url
  }

# Funcion scrape_category, toma la url de una categoria, recorre todas sus paginas y scrapea todos los libros que encuentra
def scrape_category(url):
  while True:
    print(f"Scrapeando pagina: {url}")
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Seleccionamos todos los libros en la pagina actual de esa categoría
    book_articles = soup.select('article.product_pod')

    for book in book_articles:

      # Obtenemos el href del libro
      relative_url = book.find('h3').find('a')['href']

      # relative_url trae la url completa, entonces la limpiamos para después concatenarla a base_url
      book_url = base_url + relative_url.replace('../../../', '')

      # Scrapeamos el libro
      book_data = scrape_book(book_url)

      # Guardamos el resultado
      all_books.append(book_data)

      # Pausa para evitar sobrecarga
      time.sleep(10)

    # Verificamos si hay un boton next para seguir scrapeando la sgte pagina
    # Usa un selector CSS (li.next > a) para encontrar el link de la siguiente página. Si existe: next_button va a tener un tag <a>.
    next_button = soup.select_one('li.next > a')
    if next_button:
      next_page_url = next_button['href']
      # rsplit(separador, cantidad) divide un string desde el final, y solo hace la cantidad de cortes que vos le indiques.
      url = url.rsplit('/', 1)[0] + '/' + next_page_url
    else:
      break

def scrape_all_categories():

  home_url = 'https://books.toscrape.com/index.html'
  base_site = 'https://books.toscrape.com/'

  response = requests.get(home_url)
  response.raise_for_status()
  soup = BeautifulSoup(response.text, 'html.parser')

  category_links = soup.select('div.side_categories ul li ul li a')

  print(f"\n Se encontraron {len(category_links)} categorias. \n")

  for category in category_links:
    relative_url = category['href']
    category_url = base_site + relative_url

    print(f"\n Scrapeando categoria: {category.text.strip()} -> {category_url}")

    scrape_category(category_url)

scrape_all_categories()

print(f"Total de libros scrapeados: {len(all_books)}")
print(all_books)


 Se encontraron 50 categorias. 


 Scrapeando categoria: Travel -> https://books.toscrape.com/catalogue/category/books/travel_2/index.html
Scrapeando pagina: https://books.toscrape.com/catalogue/category/books/travel_2/index.html

 Scrapeando categoria: Mystery -> https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
Scrapeando pagina: https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
❌ Error al acceder a https://books.toscrape.com/catalogue/the-past-never-ends_942/index.html: HTTPSConnectionPool(host='books.toscrape.com', port=443): Max retries exceeded with url: /catalogue/the-past-never-ends_942/index.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7d853d5832f0>, 'Connection to books.toscrape.com timed out. (connect timeout=10)'))
❌ Error al acceder a https://books.toscrape.com/catalogue/the-murder-of-roger-ackroyd-hercule-poirot-4_852/index.html: HTTPSConnectionPool(host='books.toscrape.com', port=443): 

KeyboardInterrupt: 

Para poder pasar los datos a un CSV

Un archivo CSV (valores separados por comas) es un archivo de texto que tiene un formato específico que permite guardar los datos en un formato de tabla estructurada.

with ... as f: 👉 Context manager: abre y cierra el archivo solo, incluso si explota algo. f es el manejador.

open(path, ...) 👉 ruta del archivo. Si no existe la carpeta, revienta con FileNotFoundError.

'w' 👉 write: crea el archivo o sobrescribe si ya existe.

newline='' 👉 especial para CSV en Windows: evita que se inserten líneas en blanco entre filas.

encoding='utf-8' 👉 guarda acentos/ñ/emoji sin romperse.

In [ ]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
import csv

BASE_URL = "https://books.toscrape.com/"
CATALOGUE_URL = BASE_URL + "catalogue/"

all_books = []

def fetch_author(title):
    try:
        url  = f"https://openlibrary.org/search.json?title={requests.utils.quote(title)}&limit=1"
        resp = requests.get(url, timeout=5)
        resp.raise_for_status()
        docs = resp.json().get('docs', [])
        if docs and docs[0].get('author_name'):
            return docs[0]['author_name'][0]
    except Exception as e:
        print(f"⚠️ Error al buscar autor para «{title}»: {e}")
    return "Desconocido"

def scrape_book(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"❌ Error al acceder a {url}: {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    title = soup.find('h1').text
    author = fetch_author(title)
    price = soup.select_one('.price_color').text.replace('Â', '')
    availability = soup.select_one('.availability').text.strip()

    desc_tag = soup.find('div', id='product_description')
    description = desc_tag.find_next_sibling('p').text if desc_tag else "Sin descripción"

    star_tag = soup.find('p', class_='star-rating')
    stars = star_tag['class'][1] if star_tag else "No rating"

    return {
        'Titulo': title,
        'Autor': author,
        'Precio': price,
        'Disponibilidad': availability,
        'Estrellas': stars,
        'Descripcion': description,
        'URL': url
    }

def scrape_all_books():
    url = CATALOGUE_URL + "page-1.html"
    while True:
        print(f"Scrapeando página: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        books = soup.select('article.product_pod')
        book_urls = []

        for b in books:
          # Encontramos el href del <a> dentro del <h3>
          relative_url = b.find('h3').a['href']

          # Limpiamos esa url
          cleaned_url = relative_url.replace('../../../', '')

          full_url = CATALOGUE_URL + cleaned_url

          book_urls.append(full_url)

        with ThreadPoolExecutor(max_workers = 15) as executor:
          results = executor.map(scrape_book, book_urls)
          for book_data in  results:
            if book_data:
              all_books.append(book_data)

        # Verificamos si hay un boton next para seguir scrapeando la sgte pagina
        # Usa un selector CSS (li.next > a) para encontrar el link de la siguiente página. Si existe: next_button va a tener un tag <a>.
        next_button = soup.select_one('li.next > a')
        if next_button:
          next_page = next_button['href']
          # rsplit(separador, cantidad) divide un string desde el final, y solo hace la cantidad de cortes que vos le indiques.
          url = url.rsplit('/', 1)[0] + '/' + next_page
        else:
          break

def export_books_to_csv(rows, path='books.csv'):
    """
    Guarda una lista de dicts (all_books) en un CSV.
    - Usa las claves del primer dict como encabezados.
    - Codifica en UTF-8.
    """
    if not rows:
        print("⚠️ Nada para guardar (lista vacía).")
        return
    # escribir encabezados y filas
    with open(path, 'w', newline='', encoding='utf-8') as f:
        headers = list(rows[0].keys())   # columnas: Titulo, Autor, Precio, etc.
        w = csv.DictWriter(f, fieldnames=headers)
        w.writeheader()
        w.writerows(rows)
    print(f"✅ CSV exportado: {path} ({len(rows)} filas)")


scrape_all_books()

print(f"\n Total de libros scrapeados: {len(all_books)}")

export_books_to_csv(all_books)  # crea books.csv en tu carpeta actual



Scrapeando página: https://books.toscrape.com/catalogue/page-1.html
Scrapeando página: https://books.toscrape.com/catalogue/page-2.html
Scrapeando página: https://books.toscrape.com/catalogue/page-3.html
Scrapeando página: https://books.toscrape.com/catalogue/page-4.html
Scrapeando página: https://books.toscrape.com/catalogue/page-5.html
Scrapeando página: https://books.toscrape.com/catalogue/page-6.html
Scrapeando página: https://books.toscrape.com/catalogue/page-7.html
Scrapeando página: https://books.toscrape.com/catalogue/page-8.html
Scrapeando página: https://books.toscrape.com/catalogue/page-9.html
Scrapeando página: https://books.toscrape.com/catalogue/page-10.html
Scrapeando página: https://books.toscrape.com/catalogue/page-11.html
Scrapeando página: https://books.toscrape.com/catalogue/page-12.html
Scrapeando página: https://books.toscrape.com/catalogue/page-13.html
Scrapeando página: https://books.toscrape.com/catalogue/page-14.html
Scrapeando página: https://books.toscrape.c

In [ ]:
import sqlite3

# Conectar la base de datos local 'scraping.db'
conector = sqlite3.connect('books.db')
cursor = conector.cursor()

# Crear tabla autores
cursor.execute("""
CREATE TABLE IF NOT EXISTS authors (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               name TEXT UNIQUE
               );
""")

# Crear tabla de libros
cursor.execute("""
CREATE TABLE IF NOT EXISTS books (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               title TEXT,
               price REAL,
               availability TEXT,
               stars INTEGER,
               description TEXT,
               url TEXT UNIQUE
               );
""")

# Creamos la tabla intermedia para lograr la relación de muchos a muchos
cursor.execute("""
CREATE TABLE IF NOT EXISTS books_authors(
               book_id INTEGER,
               author_id integer,
               PRIMARY KEY (book_id, author_id),
               FOREIGN KEY (book_id) REFERENCES books(id),
               FOREIGN KEY (author_id) REFERENCES authors(id)
               )
""")

# Confirmamos la creacion de las tablas
conector.commit()


In [14]:
import sqlite3

# 🔁 ABRIR NUEVA CONEXIÓN + CURSOR (si cerraste antes, hay que recrearlos)
conector = sqlite3.connect('books.db')
conector.execute("PRAGMA foreign_keys = ON;")  # que se apliquen las FKs
cursor = conector.cursor()

# Insertamos datos dentro de las tablas
# Recorremos el diccionario y vamos guardando los datos en variables
for book in all_books:
  title = book['Titulo']
  author_name = book['Autor']
  price = float(book['Precio'].lstrip('£')) # Convertimos de texto a numero
  availability = book['Disponibilidad']

  stars_text = book['Estrellas']
  star_map = {'One':1,'Two':2,'Three':3,'Four':4,'Five':5} # mapeo texto→número
  stars = star_map.get(stars_text, 0) # si no existe (ej: 'No rating'), devuelve 0

  description =  book['Descripcion']
  url = book['URL']

  # Insertar el autor
  cursor.execute(
    # ? se reemplaza por "Neil Gaiman (nombre del autor de ejemplo)" de forma segura
    "INSERT OR IGNORE INTO authors(name) VALUES(?)", (author_name,)
  )

  # Necesito el id del autor para la FK (books / book_authors).
  cursor.execute(
    "SELECT id FROM authors WHERE name = ?", (author_name,)
  )
  author_id = cursor.fetchone()[0]

  # Insertar los datos de los libros
  cursor.execute("""
    INSERT OR IGNORE INTO books (title, price, availability, stars, description, url) VALUES (?,?,?,?,?,?)
  """, (title, price, availability, stars, description, url)
  )

  # Obtenemos el id del libro
  cursor.execute(
    "SELECT id FROM books WHERE url = ?", (url,)
  )
  book_id = cursor.fetchone()[0]

  # Vinculamos libro y autor en la tabla intermedia
  cursor.execute("""
    INSERT OR IGNORE INTO books_authors(book_id, author_id) VALUES (?,?)
  """, (book_id, author_id)
  )  

# Guardamos y cerramos conexion
conector.commit()
cursor.close()
conector.close()